# Práctica 4
<br>

__Alumnos:__
* __Frederick Ernesto Borges Noronha__
* __Victor Manuel Cavero Gracia__


## PARTE 1

Implementa una función que, dados dos identificadores de ítems y una distancia
máxima, devuelva los ítems comunes que están como máximo a esa distancia de los
dos ítems iniciales.

Como Wikidata es muy grande vamos a limitarnos a buscar entidades comunes a
distancia máxima 3 usando sólo el siguiente subconjunto de propiedades:
`instance of (P31)` , `subclass of (P279)`, `has part (P527)`,
`instrument (P1303)`, `genre (P136)`

Si no hay entidades comunes a esa distancia diremos que los ítems no están
relacionados.

Calcula las relaciones entre las siguientes entidades dos a dos y explica en lenguaje
natural el tipo de relación encontrada:
`piano (Q5994)`, `electronic keyboard (Q1343007)`,
`String synthesizer (Q2355465)`, `Hans Zimmer (Q76364)`,
`Pirates of the Caribbean: The Curse of the Black Pearl (Q46717)`,
`The Lion King (Q36479)`, `Toy Story (Q171048)`, `Iron Man (Q192724)`.

- Consejos:
  - Obtener la información de una entidad Wikidata con get_entity_dict_from_api es
    una operación lenta. Te recomendamos que crees una cache de entidades (map
    entidad_id -> entidad) que evite pedir una y otra vez las mismas entidades a
    Wikidata.
  - Te recomendamos crear una función get_claims(item_id) que devuelva las
    relaciones que salen a partir de una entidad en formato (id_propiedad, id_valor).
    Sólo debes considerar los truthy_claims que tengan como valor un item de
    Wikidata (entidades que empiezan por ‘Q’). Recuerda que también hemos
    limitado las propiedades que debes considerar.
  - Es importante tener en cuenta que, cuando expandimos una onda, la intersección
    se puede producir con cualquiera de las entidades de la otra onda (y no sólo con
    las de la frontera).


In [1]:
from qwikidata.entity import WikidataItem, WikidataProperty, WikidataLexeme 
from qwikidata.linked_data_interface import get_entity_dict_from_api


def items_comunes(item_id1, item_id2, distancia_maxima=3):
    if distancia_maxima > 3 : distancia_maxima = 3
    dict_cache = dict() # Caché de la forma entidad_id -> entidad
    items_comunes = [] # Lista con (entidad_id,distancia)
    items_insertados = dict() # Eliminar repetidos 
    
    dist1_id1 = get_claims(item_id1, dict_cache) # Distancia de 1 a item_id1
    dist1_id2 = get_claims(item_id2, dict_cache) # Distancia de 1 a item_id2
    dic_dist2_id1 = dict() # Distancia de 2 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    dic_dist2_id2 = dict() # Distancia de 2 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    
    for pair in dist1_id1:
        dic_dist2_id1[pair] = get_claims(pair[1], dict_cache)

    for pair in dist1_id2:
        dic_dist2_id2[pair] = get_claims(pair[1], dict_cache)
    
    for pair_id1 in dist1_id1:
        for pair_id2 in dist1_id2:
            if(pair_id1[1] == pair_id2[1]):
                items_comunes.append((pair_id2[1],2))
            for pair2_id1 in dic_dist2_id1[pair_id1]:
                if(pair2_id1[1] == pair_id2[1]):
                    items_comunes.append((pair_id2[1],3))
            for pair2_id2 in dic_dist2_id2[pair_id2]:
                if(pair2_id2[1] == pair_id1[1]):
                    items_comunes.append((pair2_id2[1],3))
    
    cadena = '--- ENTIDAD: {} , NOMBRE {} Y DISTANCIA: {} '
    
    for item in items_comunes:
        if(item[1] <= distancia_maxima):
            poner = True
            
            if item[0] in items_insertados : 
                if item[1] == items_insertados[item[0]] : poner = False
            
            if poner :
                if item[0] in dict_cache : print(cadena.format(item[0], dict_cache[item[0]].get_label(), item[1]))
                else :
                    item_dict = get_entity_dict_from_api(item[0])
                    entidad = WikidataItem(item_dict)
                    print(cadena.format(item[0], entidad.get_label(), item[1]))
                
                items_insertados[item[0]] = item[1]
    
def get_claims(item_id, dict_cache):
    # truthly_claims que empiezen por 'Q' id_valor y id_propiedad sea : P31, P279, P527, P1303 o P136
    
    tuple_list = []
    propiedades = ['P31','P279','P527','P1303','P136']

    if item_id in dict_cache:
        entidad = dict_cache[item_id]
    else:    
        item_dict = get_entity_dict_from_api(item_id)
        entidad = WikidataItem(item_dict)
        dict_cache[item_id] = entidad
    
    for x in propiedades:
        claim_group = entidad.get_truthy_claim_group(x)
        for y in claim_group:
            tuple_list.append((x,y.mainsnak.datavalue.value['id']))
            
    return tuple_list # (id_propiedad, id_valor)
            

Calcular las relaciones entre las siguientes entidades dos a dos y explica en lenguaje
natural el tipo de relación encontrada:

In [2]:
items_comunes('Q5994','Q1343007')

--- ENTIDAD: Q34379 , NOMBRE musical instrument Y DISTANCIA: 3 
--- ENTIDAD: Q52954 , NOMBRE keyboard instrument Y DISTANCIA: 2 


In [3]:
items_comunes('Q2355465','Q76364')

--- ENTIDAD: Q1327500 , NOMBRE electronic musical instrument Y DISTANCIA: 3 


In [4]:
items_comunes('Q46717','Q36479')

--- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3 


In [5]:
items_comunes('Q171048','Q192724')

--- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3 


__Las relaciones encontradas se explican en la siguiente parte__ donde vemos con exactitud los caminos.

## PARTE 2

Implementa una versión avanzada del algoritmo visto en clase que no sólo devuelva
las entidades comunes sino también los caminos que conectan cada una de las
entidades originales con esas entidades comunes. En este caso, la función sólo debe
devolver los caminos de longitud mínima. Si no hay caminos de longitud menor o igual
a 5 diremos que las entidades no están relacionadas.

- Consejos:
    - La intersección de las dos ondas puede contener varias entidades. Debes
        considerar todas ellas a la hora de calcular las posibles soluciones. Además, cada
        una de las entidades comunes puede dar lugar a varias soluciones si hay distintos
        caminos que conectan las entidades iniciales con esa entidad común.
    - De todas las soluciones posibles sólo debes devolver las de longitud mínima,
        entendiendo que la longitud de la solución es la suma de las longitudes de los
        caminos desde las entidades iniciales hasta la entidad común.
    - Hay muchas formas de implementar este algoritmo. Una de ellas consiste en
        almacenar todos los caminos de cada una de las ondas e ir expandiéndolos
        alternativamente hasta que las ondas intersequen. Para calcular la intersección
        deberás almacenar también los ítems de cada una de las ondas.
    - Ten en cuenta que para calcular todas las posibles soluciones de longitud <= n
        debes expandir cada una de las ondas n veces (puede que la relación vaya del
        primer ítem hasta el segundo).

In [6]:
from qwikidata.entity import WikidataItem, WikidataProperty, WikidataLexeme 
from qwikidata.linked_data_interface import get_entity_dict_from_api


def camino_minimo(item_id1, item_id2):
    dict_cache = dict() # Caché de la forma entidad_id -> entidad
    items_comunes = dict() # (item_comun, ditancia) = [entidad_ id] con el camino asociado
    items_camino_minimo = dict() # item_comun = distancia minima
    
    dist1_id1 = get_claims(item_id1, dict_cache) # Distancia de 1 a item_id1
    dist1_id2 = get_claims(item_id2, dict_cache) # Distancia de 1 a item_id2
    dic_dist2_id1 = dict() # Distancia de 2 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    dic_dist2_id2 = dict() # Distancia de 2 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    dic_dist3_id1 = dict() # Distancia de 3 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    dic_dist3_id2 = dict() # Distancia de 3 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    
    for pair in dist1_id1:
        dic_dist2_id1[pair] = get_claims(pair[1], dict_cache)

    for pair in dist1_id2:
        dic_dist2_id2[pair] = get_claims(pair[1], dict_cache)
        
    for pair in dic_dist2_id1.values():
        for x in pair:
            dic_dist3_id1[x] = get_claims(x[1], dict_cache)
        
    for pair in dic_dist2_id2.values():
        for x in pair:
            dic_dist3_id2[x] = get_claims(x[1], dict_cache)
    
    # Al camino añadido al diccionario item_comunes le falta item_id1 inicial y item_id2 final
    
    for pair_id1 in dist1_id1:
        for pair_id2 in dist1_id2:
            for pair2_id1 in dic_dist2_id1[pair_id1]:
                for pair2_id2 in dic_dist2_id2[pair_id2]:
                    
                    if(pair_id1[1] == pair_id2[1]):
                        items_comunes[(pair_id2[1],2)] = [pair_id1, pair_id2]
                    if(pair2_id2[1] == pair_id1[1]):
                        items_comunes[(pair2_id2[1],3)] = [pair_id1, pair2_id2, pair_id2]
                    if(pair2_id1[1] == pair_id2[1]):
                        items_comunes[(pair_id2[1],3)] = [pair_id1, pair2_id1, pair_id2]
                    if(pair2_id1[1] == pair2_id2[1]):
                        items_comunes[(pair2_id2[1],4)] = [pair_id1, pair2_id1, pair2_id2, pair_id2]
                        
                    for pair3_id1 in dic_dist3_id1[pair2_id1]:
                        if(pair3_id1[1] == pair_id2[1]):
                            items_comunes[(pair_id2[1],4)] = [pair_id1, pair2_id1, pair3_id1, pair_id2]
                        if(pair3_id1[1] == pair2_id2[1]):
                            items_comunes[(pair2_id2[1],5)] = [pair_id1, pair2_id1, pair3_id1, pair2_id2, pair_id2]
                            
                    for pair3_id2 in dic_dist3_id2[pair2_id2]:
                        if(pair3_id2[1] == pair_id1[1]):
                            items_comunes[(pair_id1[1],4)] = [pair_id1, pair3_id2, pair2_id2, pair_id2] 
                        if(pair3_id2[1] == pair2_id1[1]):
                            items_comunes[(pair2_id1[1],5)] = [pair_id1, pair2_id1, pair3_id2, pair2_id2, pair_id2]
    
    cadena = ' --- ENTIDAD: {} , NOMBRE {} Y DISTANCIA: {}, CAMINO GENERADO: {} '
    
    for item in items_comunes.keys():
        poner = True
        lista = items_comunes[item]
        lista.insert(0, item_id1)
        lista.append(item_id2)
        
        if item[0] in items_camino_minimo:
            if item[1] > items_camino_minimo[item[0]] : poner = False
        
        if poner:
            if item[0] in dict_cache : print(cadena.format(item[0], dict_cache[item[0]].get_label(), 
                                                           item[1], lista))
            else :
                item_dict = get_entity_dict_from_api(item[0])
                entidad = WikidataItem(item_dict)
                print(cadena.format(item[0], entidad.get_label(), item[1], item_id1 + items_comunes[item] + item_id2))
                
            items_camino_minimo[item[0]] = item[1] 
    
            

Comparación de las entidades del apartado anterior dos a dos mostrando los
caminos de longitud mínima. Considerando las propiedades indicadas
previamente

In [7]:
camino_minimo('Q5994','Q1343007')

 --- ENTIDAD: Q34379 , NOMBRE musical instrument Y DISTANCIA: 3, CAMINO GENERADO: ['Q5994', ('P31', 'Q34379'), ('P279', 'Q34379'), ('P279', 'Q52954'), 'Q1343007'] 
 --- ENTIDAD: Q267228 , NOMBRE sound generator Y DISTANCIA: 5, CAMINO GENERADO: ['Q5994', ('P31', 'Q34379'), ('P279', 'Q267228'), ('P279', 'Q267228'), ('P279', 'Q34379'), ('P279', 'Q52954'), 'Q1343007'] 
 --- ENTIDAD: Q39546 , NOMBRE tool Y DISTANCIA: 5, CAMINO GENERADO: ['Q5994', ('P31', 'Q34379'), ('P279', 'Q39546'), ('P279', 'Q39546'), ('P279', 'Q34379'), ('P279', 'Q52954'), 'Q1343007'] 
 --- ENTIDAD: Q19659292 , NOMBRE musical instrument part Y DISTANCIA: 5, CAMINO GENERADO: ['Q5994', ('P31', 'Q34379'), ('P527', 'Q19659292'), ('P279', 'Q19659292'), ('P527', 'Q901207'), ('P279', 'Q52954'), 'Q1343007'] 
 --- ENTIDAD: Q1254773 , NOMBRE class of instruments Y DISTANCIA: 5, CAMINO GENERADO: ['Q5994', ('P279', 'Q4951628'), ('P279', 'Q55724742'), ('P31', 'Q1254773'), ('P31', 'Q1254773'), ('P279', 'Q1327500'), 'Q1343007'] 
 --- 

Las dos entidades están relacionadas mediante la entidad `musical instrument Q34379` de manera que `piano Q5994` es instancia de `musical instrument Q34379` (la entidad común) y `electronic keyboard Q1343007` es subclase de `keyboard instrument Q52954` que a su vez es subclase de  `musical instrument Q34379`.

Las dos entidades están relacionadas mediante la entidad `keyboard instrument Q52954` de modo que `piano Q5994` es subclase de `keyboard instrument Q52954` y `electronic keyboard Q1343007` es subclase de `keyboard instrument Q52954`.

In [8]:
camino_minimo('Q2355465','Q76364')

 --- ENTIDAD: Q1254773 , NOMBRE class of instruments Y DISTANCIA: 5, CAMINO GENERADO: ['Q2355465', ('P31', 'Q1327500'), ('P31', 'Q1254773'), ('P31', 'Q1254773'), ('P279', 'Q57306162'), ('P1303', 'Q6607'), 'Q76364'] 
 --- ENTIDAD: Q34379 , NOMBRE musical instrument Y DISTANCIA: 5, CAMINO GENERADO: ['Q2355465', ('P31', 'Q1327500'), ('P279', 'Q105738'), ('P279', 'Q34379'), ('P279', 'Q34379'), ('P1303', 'Q52954'), 'Q76364'] 
 --- ENTIDAD: Q1254773 , NOMBRE class of instruments Y DISTANCIA: 4, CAMINO GENERADO: ['Q2355465', ('P31', 'Q1327500'), ('P31', 'Q1254773'), ('P31', 'Q1254773'), ('P1303', 'Q52954'), 'Q76364'] 
 --- ENTIDAD: Q1327500 , NOMBRE electronic musical instrument Y DISTANCIA: 3, CAMINO GENERADO: ['Q2355465', ('P31', 'Q1327500'), ('P279', 'Q1327500'), ('P1303', 'Q163829'), 'Q76364'] 
 --- ENTIDAD: Q105738 , NOMBRE electrophone Y DISTANCIA: 5, CAMINO GENERADO: ['Q2355465', ('P31', 'Q1327500'), ('P279', 'Q105738'), ('P279', 'Q105738'), ('P279', 'Q1327500'), ('P1303', 'Q163829'), 

Las dos entidades están relacionadas mediante la entidad `electronic musical instrument Q1327500` de modo que `string synthesizer Q2355465` es instancia de `electronic musical instrument Q1327500` y  `hans zimmer Q76364` su instrumento es `keyboard instrument Q52954` que a su vez es subclase de `electronic musical instrument Q1327500`.

In [9]:
camino_minimo('Q46717','Q36479')

 --- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3, CAMINO GENERADO: ['Q46717', ('P31', 'Q11424'), ('P279', 'Q11424'), ('P136', 'Q130232'), 'Q36479'] 
 --- ENTIDAD: Q4502142 , NOMBRE visual artwork Y DISTANCIA: 5, CAMINO GENERADO: ['Q46717', ('P31', 'Q11424'), ('P279', 'Q4502142'), ('P279', 'Q4502142'), ('P279', 'Q11424'), ('P136', 'Q130232'), 'Q36479'] 
 --- ENTIDAD: Q2431196 , NOMBRE audiovisual work Y DISTANCIA: 5, CAMINO GENERADO: ['Q46717', ('P31', 'Q11424'), ('P279', 'Q2431196'), ('P279', 'Q2431196'), ('P279', 'Q11424'), ('P136', 'Q130232'), 'Q36479'] 
 --- ENTIDAD: Q20937557 , NOMBRE series Y DISTANCIA: 5, CAMINO GENERADO: ['Q46717', ('P31', 'Q11424'), ('P279', 'Q20937557'), ('P279', 'Q20937557'), ('P279', 'Q11424'), ('P136', 'Q130232'), 'Q36479'] 
 --- ENTIDAD: Q10301427 , NOMBRE moving image Y DISTANCIA: 5, CAMINO GENERADO: ['Q46717', ('P31', 'Q11424'), ('P279', 'Q10301427'), ('P279', 'Q10301427'), ('P279', 'Q11424'), ('P136', 'Q130232'), 'Q36479'] 
 --- ENTIDAD: Q20820424 , NO

Las dos entidades están relacionadas mediante la entidad `film Q11424` de modo que `Pirates of the Caribbean: The Curse of the Black Pearl Q46717` es instancia de `film Q11424` y  `The Lion King Q36479` es del género `drama Q130232` que a su vez esta entidad es sublcase de `film Q11424` (entidad común).

In [10]:
camino_minimo('Q171048','Q192724')

 --- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 4, CAMINO GENERADO: ['Q171048', ('P136', 'Q157394'), ('P279', 'Q2973181'), ('P279', 'Q11424'), ('P31', 'Q11424'), 'Q192724'] 
 --- ENTIDAD: Q201658 , NOMBRE film genre Y DISTANCIA: 5, CAMINO GENERADO: ['Q171048', ('P136', 'Q157394'), ('P279', 'Q2973181'), ('P31', 'Q201658'), ('P31', 'Q201658'), ('P136', 'Q1535153'), 'Q192724'] 
 --- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3, CAMINO GENERADO: ['Q171048', ('P136', 'Q663106'), ('P279', 'Q11424'), ('P31', 'Q11424'), 'Q192724'] 
 --- ENTIDAD: Q4502142 , NOMBRE visual artwork Y DISTANCIA: 5, CAMINO GENERADO: ['Q171048', ('P136', 'Q663106'), ('P279', 'Q11424'), ('P279', 'Q4502142'), ('P279', 'Q4502142'), ('P31', 'Q11424'), 'Q192724'] 
 --- ENTIDAD: Q2431196 , NOMBRE audiovisual work Y DISTANCIA: 5, CAMINO GENERADO: ['Q171048', ('P136', 'Q663106'), ('P279', 'Q11424'), ('P279', 'Q2431196'), ('P279', 'Q2431196'), ('P31', 'Q11424'), 'Q192724'] 
 --- ENTIDAD: Q20937557 , NOMBRE series Y DISTANCIA:

Las dos entidades están relacionadas mediante la entidad `film Q11424` de modo que `Toy Story Q171048` es del género `buddy film Q663106` que a la vez es subclase de `film Q11424` y por la otra parte `Iron Man Q192724` es instancia de `film Q11424` la entidad común.